In [1]:
from ProjectV2.comment import Comment
import ProjectV2.get_texts as GetTexts
import keras 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


Using TensorFlow backend.


In [4]:
import csv
def readFile(filename):
    sarc_comments = []
    neg_comments = []
    
    stops = set(stopwords.words('english'))
    #Open CSV file and read each line
    with open(filename, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        wordnet_lem = WordNetLemmatizer()
        for row in reader:
            sentence = row['comment']
            if len(sentence.split())>3:
                clean=[]
                for word in word_tokenize(sentence):
                    if word not in stops:
                        clean.append(wordnet_lem.lemmatize(word))
                if(row['label']=='1'):
                    sarc_comments.append(clean)
                elif(row['label']=='0'):
                    neg_comments.append(clean)
     #If both arrays have elements save both into np arrays so they can be easier accessed for future uses and return both
    if len(sarc_comments) >0 and len(neg_comments) >0:
        # Return both sets of comments
        return sarc_comments, neg_comments

    #If only sarcastic comments then save this and return them
    elif len(sarc_comments) >0:
        #return sarc_comments
        return sarc_comments

    #If only non sarcastic then save and return them
    elif len(neg_comments) >0:
        #return neg_comments
        return neg_comments

In [12]:
sarcComments = readFile("ProjectV2/Sarc Set.csv")
negComments = readFile("ProjectV2/Non Sarc Set.csv")

In [16]:
print(sarcComments[1000])

['You', 'forgot', '-']


In [18]:
combined =[]
for comment in sarcComments:
    combined.append([1, comment])
for comment in negComments:
    combined.append([0, comment])

In [19]:
print(combined[0])

[1, ['But', "'ll", 'review', '!']]


In [24]:
import pandas as pd

dataframe = pd.DataFrame(combined, columns=("label","comment"))

In [25]:
print(dataframe.head(20))

    label                                            comment
0       1                              [But, 'll, review, !]
1       1  [wow, totally, unreasonable, assume, agency, c...
2       1  [Ho, ho, ho, ..., But, Melania, said, way, cou...
3       1  [I, ca, n't, wait, @, potus, start, twitter, w...
4       1  [got, ta, love, teacher, give, exam, day, hall...
5       1  [Hoiberg, said, ``, great, player, '', ,, Dwight]
6       1  [Oh, ,, I, never, realized, easy, ,, I, ,, eve...
7       1                [Exactly, ,, reason, whatsoever, .]
8       1   [Hank, would, great, time, Legends, Tomorrow, .]
9       1  [I, remember, reading, story, news, riot, poli...
10      1  [ca, n't, watch, get, heaven, son, ,, 's, stea...
11      1  [Yeah, ,, kid, ca, n't, eat, peanut, without, ...
12      1       [Dude, heard, Lions, legend, Armbrisster, ?]
13      1  [I, mean, guy, still, n't, understand, I, n't,...
14      1                             [How, Marcos, Rojo, ?]
15      1  [,, Galio, st

In [27]:
dataframe = dataframe.sample(frac=1).reset_index(drop=True)

In [28]:
print(dataframe.head(20))

    label                                            comment
0       0                   [He, cover, strategic, beard, .]
1       0                                     [thanks, :, )]
2       1                       [Says, guy, n't, married, .]
3       1                                 [I, think, forgot]
4       0  [Sorry, n't, round, yet, ,, I, 'm, still, expl...
5       1      [totally, sound, like, rejecting, advance, .]
6       1  [Yeah, ,, saying, woman, 'blacks, ', uninforme...
7       1                            [You, 'll, never, know]
8       0                 [yes, ,, seen, kv-2, lob, shot, ?]
9       1            [He, 's, definitely, puppet, though, .]
10      1  [Yeah, man, ,, semblance, sexuality, visible, ...
11      0           [wenigstens, mal, keine, gelben, briefe]
12      0                                  [Is, dad, cat, ?]
13      1  [Because, anarcho, capitalism, course, actual,...
14      1                     [And, woman, mostly, whore, .]
15      1  [Sure, ,, coa

In [30]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.model_selection import train_test_split

In [31]:
train, test = train_test_split(dataframe, test_size=0.2)

In [32]:
len(train), len(test)

(39998, 10000)

In [33]:
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train['comment'])
 
x_train = tokenizer.texts_to_matrix(train['comment'], mode='tfidf')
x_test = tokenizer.texts_to_matrix(test['comment'], mode='tfidf') 

In [59]:
from sklearn.preprocessing import LabelBinarizer


encoder = LabelBinarizer()
encoder.fit(train['label'])
y_train = encoder.transform(train['label'])
y_test = encoder.transform(test['label'])


In [63]:
print(y_train[100])

[1]


In [58]:
print (x_train[7:13])

[[0.         1.30951992 1.51457641 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.51457641 ... 0.         0.         0.        ]]


In [64]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               7680512   
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_6 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 513       
Total para

Train on 35998 samples, validate on 4000 samples
Epoch 1/30


  100/35998 [..............................] - ETA: 15:17 - loss: 0.6893 - acc: 0.5100

  200/35998 [..............................] - ETA: 8:26 - loss: 0.6790 - acc: 0.5650 

  300/35998 [..............................] - ETA: 6:12 - loss: 0.6849 - acc: 0.5533

...

35600/35998 [============================>.] - ETA: 1s - loss: 0.0291 - acc: 0.9873

35700/35998 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.9873

35800/35998 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.9872

35900/35998 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.9872

35998/35998 [==============================] - 101s 3ms/step - loss: 0.0292 - acc: 0.9872 - val_loss: 2.9305 - val_acc: 0.6525


In [66]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])
 
# text_labels = encoder.classes_
#  
# for i in range(10):
#     prediction = model.predict(np.array([x_test[i]]))
#     predicted_label = text_labels[np.argmax(prediction[0])]
#     print(test_files_names.iloc[i])
#     print('Actual label:' + test_tags.iloc[i])
#     print("Predicted label: " + predicted_label)

  100/10000 [..............................] - ETA: 5s

  200/10000 [..............................] - ETA: 5s

  300/10000 [..............................] - ETA: 6s

  400/10000 [>.............................] - ETA: 6s

  500/10000 [>.............................] - ETA: 6s

...

 9600/10000 [===========================>..] - ETA: 0s

 9800/10000 [============================>.] - ETA: 0s



10000/10000 [==============================] - 6s 608us/step


Test accuracy: 0.6462000012397766


In [1]:
import keras 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [6]:

filename = ("ProjectV2/MegaSet.csv")
sarcComments, negComments = readFile(filename)

In [7]:
combined =[]
for comment in sarcComments:
    combined.append([1, comment])
for comment in negComments:
    combined.append([0, comment]) 
    
dataframe = pd.DataFrame(combined, columns=("label","comment"))

dataframe = dataframe.sample(frac=1).reset_index(drop=True)

train, test = train_test_split(dataframe, test_size=0.2)

len(train), len(test)

(67455, 16864)

In [12]:
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train['comment'])
 
x_train = tokenizer.texts_to_matrix(train['comment'], mode='tfidf')
x_test = tokenizer.texts_to_matrix(test['comment'], mode='tfidf') 

encoder = LabelBinarizer()
encoder.fit(train['label'])
y_train = encoder.transform(train['label'])
y_test = encoder.transform(test['label'])


MemoryError: 

In [9]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total para

NameError: name 'y_train' is not defined

In [11]:
print(y_train[10])

NameError: name 'y_train' is not defined